In [ ]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/anayap0/strategyqa_v2.git

Cloning into 'strategyqa_v2'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 70 (delta 33), reused 52 (delta 18), pack-reused 0
Receiving objects: 100% (70/70), 25.82 MiB | 15.84 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', DEVICE)

Device: cuda


In [ ]:
from strategyqa_v2.src.SQP1Dataset import initialize_datasets, SQP1Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer, T5Model, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Optimizer, AdamW
from tqdm.notebook import tqdm

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained('t5-base').to(DEVICE)

datasets = initialize_datasets('strategyqa_v2/data/train.json', 'strategyqa_v2/data/dev.json', tokenizer)
print(datasets['train'][0])
train_dataloader = DataLoader(datasets['train'],
                                   batch_size=32,
                                   shuffle=False,
                                   collate_fn=SQP1Dataset.collate_fn)

validation_dataloader = DataLoader(datasets['dev'],
                                   batch_size=32,
                                   shuffle=False,
                                   collate_fn=SQP1Dataset.collate_fn)

# print(validation_dataloader)
# for data in validation_dataloader:
#   print(data)
batch = next(iter(validation_dataloader))

print(f"{len(datasets['dev'])}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


SQP1Example(question='Are more people today related to Genghis Khan than Julius Caesar?', decompositions=['How many kids did Julius Caesar have?', 'How many kids did Genghis Khan have?', 'Is #2 greater than #1?'])
229


In [ ]:
#### THIS WORKS WITHOUT ERRORS
input_question = "Are more people today related to Genghis Khan than Julius Caesar?"
decompositions = [
            "How many kids did Julius Caesar have?",
            "How many kids did Genghis Khan have?",
            "Is #2 greater than #1?"
        ]

inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).input_ids.to(DEVICE)
outputs = tokenizer("<SEP>".join(decompositions), return_tensors="pt", padding=True, truncation=True).input_ids.to(DEVICE)
print(inputs)
print(outputs)

model.train()
# Fine-tuning
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
for epoch in range(5):
    optimizer.zero_grad()
    oputs = model(input_ids=inputs, labels=outputs)
    loss = oputs.loss
    loss.backward()
    optimizer.step()

# evaluate model after training on one example 5 times
model.eval()
predictions = model.generate(input_ids=inputs, max_length=512)
print(tokenizer.decode(predictions[0], skip_special_tokens=True))


tensor([[ 1521,    72,   151,   469,  1341,    12,  5945,  5649,     7, 14420,
           145,  9983,   302, 26218,    58,     1]], device='cuda:0')
tensor([[  571,   186,  1082,   410,  9983,   302, 26218,    43,    58,     2,
           134,  8569,  3155,  7825,   186,  1082,   410,  5945,  5649,     7,
         14420,    43,    58,     2,   134,  8569,  3155,   196,     7, 15493,
          2123,   145,  7172,    58,     1]], device='cuda:0')
Are more people related to Genghis Khan than Julius Caesar?


In [ ]:
def train_one_epoch(model: nn.Module, train_dataloader: DataLoader, optimizer: Optimizer, epoch: int):

    model.train()
    with tqdm(train_dataloader, desc=f"Train Ep {epoch}", total=len(train_dataloader)) as tq:
        for batch in tq:
            inputs = batch['input_ids'].input_ids.to(model.device)
            outputs = batch['target_ids'].input_ids.to(model.device)

            loss = model(input_ids=inputs, labels=outputs).loss
            print(f"loss at epoch {epoch}: {loss}")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


In [ ]:
def evaluate(model: nn.Module, dataloader: DataLoader):
  model.eval()
  all_predictions = []
  with torch.no_grad():
    with tqdm(dataloader, desc=f"Train Ep {epoch}", total=len(train_dataloader)) as tq:
      for batch in tq:
        inputs = batch['input_ids'].input_ids.to(DEVICE)
        predictions = model.generate(input_ids=inputs, max_length=512)
        all_predictions += predictions
        # text predictions

        # print(tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions)

  return all_predictions


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
for i in range(1, 11):
  train_one_epoch(model, train_dataloader, optimizer, i)

Train Ep 1:   0%|          | 0/65 [00:00<?, ?it/s]

loss at epoch 1: 9.07058334350586
loss at epoch 1: 7.698611259460449
loss at epoch 1: 5.025505542755127
loss at epoch 1: 4.127199649810791
loss at epoch 1: 3.6759033203125
loss at epoch 1: 4.319543838500977
loss at epoch 1: 3.559028387069702
loss at epoch 1: 2.4993107318878174
loss at epoch 1: 2.776299476623535
loss at epoch 1: 2.445887565612793
loss at epoch 1: 2.60064959526062
loss at epoch 1: 2.539801836013794
loss at epoch 1: 2.5269269943237305
loss at epoch 1: 2.420578718185425
loss at epoch 1: 2.1620712280273438
loss at epoch 1: 2.3524487018585205
loss at epoch 1: 2.583688974380493
loss at epoch 1: 2.518007755279541
loss at epoch 1: 2.2333505153656006
loss at epoch 1: 1.8560748100280762
loss at epoch 1: 1.88676917552948
loss at epoch 1: 2.6253092288970947
loss at epoch 1: 2.1314871311187744
loss at epoch 1: 2.212197780609131
loss at epoch 1: 2.075395107269287
loss at epoch 1: 1.967165231704712
loss at epoch 1: 2.0579047203063965
loss at epoch 1: 1.681607723236084
loss at epoch 1:

Train Ep 2:   0%|          | 0/65 [00:00<?, ?it/s]

loss at epoch 2: 1.1811189651489258
loss at epoch 2: 0.9946280717849731
loss at epoch 2: 1.0160892009735107
loss at epoch 2: 1.3274177312850952
loss at epoch 2: 1.2187143564224243
loss at epoch 2: 1.2638212442398071
loss at epoch 2: 1.2163491249084473
loss at epoch 2: 0.887388288974762
loss at epoch 2: 1.1568522453308105
loss at epoch 2: 1.0500355958938599
loss at epoch 2: 1.3594316244125366
loss at epoch 2: 1.0485914945602417
loss at epoch 2: 1.123901128768921
loss at epoch 2: 1.2122315168380737
loss at epoch 2: 0.9155861735343933
loss at epoch 2: 1.2840981483459473
loss at epoch 2: 1.3373384475708008
loss at epoch 2: 1.3405290842056274
loss at epoch 2: 1.1055160760879517
loss at epoch 2: 1.0900847911834717
loss at epoch 2: 0.9669348001480103
loss at epoch 2: 1.6096649169921875
loss at epoch 2: 1.270062804222107
loss at epoch 2: 1.3169846534729004
loss at epoch 2: 1.2320020198822021
loss at epoch 2: 1.099848747253418
loss at epoch 2: 1.2948907613754272
loss at epoch 2: 1.0319170951843

Train Ep 3:   0%|          | 0/65 [00:00<?, ?it/s]

...

loss at epoch 8: 0.7336783409118652
loss at epoch 8: 0.662996232509613
loss at epoch 8: 0.6432072520256042
loss at epoch 8: 0.839701235294342
loss at epoch 8: 0.8355404734611511
loss at epoch 8: 0.7619725465774536
loss at epoch 8: 0.8161048293113708
loss at epoch 8: 0.5889711380004883
loss at epoch 8: 0.7636610865592957
loss at epoch 8: 0.7205092906951904
loss at epoch 8: 0.9526306986808777
loss at epoch 8: 0.68963623046875
loss at epoch 8: 0.7595347762107849
loss at epoch 8: 0.7771540284156799
loss at epoch 8: 0.584252119064331
loss at epoch 8: 0.8465445041656494
loss at epoch 8: 0.8928321003913879
loss at epoch 8: 0.9335652589797974
loss at epoch 8: 0.7350932955741882
loss at epoch 8: 0.7676220536231995
loss at epoch 8: 0.5728126764297485
loss at epoch 8: 1.1413342952728271
loss at epoch 8: 0.8691628575325012
loss at epoch 8: 0.9313428401947021
loss at epoch 8: 0.841550350189209
loss at epoch 8: 0.7437366247177124
loss at epoch 8: 0.9212163090705872
loss at epoch 8: 0.733605086803436

Train Ep 9:   0%|          | 0/65 [00:00<?, ?it/s]

loss at epoch 9: 0.7054017782211304
loss at epoch 9: 0.6508780717849731
loss at epoch 9: 0.6160632967948914
loss at epoch 9: 0.811367928981781
loss at epoch 9: 0.8215184211730957
loss at epoch 9: 0.7300683259963989
loss at epoch 9: 0.7657039165496826
loss at epoch 9: 0.5751426815986633
loss at epoch 9: 0.7398453950881958
loss at epoch 9: 0.6963361501693726
loss at epoch 9: 0.9514893889427185
loss at epoch 9: 0.6708401441574097
loss at epoch 9: 0.7256298661231995
loss at epoch 9: 0.7666141986846924
loss at epoch 9: 0.5679004192352295
loss at epoch 9: 0.841062605381012
loss at epoch 9: 0.865411102771759
loss at epoch 9: 0.8882833123207092
loss at epoch 9: 0.6944157481193542
loss at epoch 9: 0.7517615556716919
loss at epoch 9: 0.556676983833313
loss at epoch 9: 1.0792620182037354
loss at epoch 9: 0.8405061364173889
loss at epoch 9: 0.8876953721046448
loss at epoch 9: 0.8203960657119751
loss at epoch 9: 0.7030701041221619
loss at epoch 9: 0.894467830657959
loss at epoch 9: 0.71985369920730

Train Ep 10:   0%|          | 0/65 [00:00<?, ?it/s]

loss at epoch 10: 0.6874386072158813
loss at epoch 10: 0.6322190761566162
loss at epoch 10: 0.6074788570404053
loss at epoch 10: 0.7930088043212891
loss at epoch 10: 0.8006817698478699
loss at epoch 10: 0.7167069315910339
loss at epoch 10: 0.762771487236023
loss at epoch 10: 0.5514660477638245
loss at epoch 10: 0.733320415019989
loss at epoch 10: 0.6972560286521912
loss at epoch 10: 0.8938800692558289
loss at epoch 10: 0.6533885598182678
loss at epoch 10: 0.6953824758529663
loss at epoch 10: 0.735884428024292
loss at epoch 10: 0.54905104637146
loss at epoch 10: 0.7815907001495361
loss at epoch 10: 0.85650235414505
loss at epoch 10: 0.8765488266944885
loss at epoch 10: 0.6889231204986572
loss at epoch 10: 0.7269969582557678
loss at epoch 10: 0.5359954237937927
loss at epoch 10: 1.0764018297195435
loss at epoch 10: 0.8114355206489563
loss at epoch 10: 0.8387885689735413
loss at epoch 10: 0.7813063859939575
loss at epoch 10: 0.7053109407424927
loss at epoch 10: 0.8679310083389282
loss at 

In [ ]:
preds = evaluate(model, validation_dataloader)

for pred in preds:
  print(tokenizer.decode(pred, skip_special_tokens=True))

Train Ep 4:   0%|          | 0/65 [00:00<?, ?it/s]

What is the population of the Albany in Georgia?SEP>How many people are in New York?SEP>Is #1 greater than or equal to #2?
What language is used in Saint Vincent and the Grenadines?SEP>What language is English spoken in?SEP>Is #1 the same as #2?
What is greed?SEP>What are the Seven Deadly Sins?SEP>Is #1 the most prevalent?
What is Mount Fuji's topography?SEP>What is the topography of the Sea of Japan?SEP>Is #2 greater than or equal to #1?
What is Lil Jon's top ranked song on Billboard?SEP>What is The Lox's biggest hit?SEP>Is #1 the same as #2?
What is the population of Miami?SEP>What is the population of the American West Coast?SEP>Is #2 greater than or equal to #1?
What is the number of members of the Virginia General Assembly?SEP>What is the number of members of the Swiss Guard?SEP>Is #1 greater than or equal to #2?
What was the role of Switzerland in WWII?SEP>What was the role of Portugal in the Portuguese Colonial War?SEP>Is #2 the same as #2?
What language is Old English?SEP>What 

In [22]:
!git add -A

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968